# RDS 인증과 접속

## Amazon RDS 접속
- 생성 시 두 종류의 IP가 할당
    - Private IP : 기본적으로 할당
        - VPC 내부의 리소스가 RDS에 접근하기 위해서 사용
        - RDS의 DB 인스턴스가 위치한 서브넷에 따라 Range 결정
    - Public IP : 퍼블릭 접근 가능 옵션을 선택했을 경우 할당
        - 해당 RDS 인스턴스가 Private 서브넷에 위치한 경우 할당 되지 않음
- RDS의 IP는 다양한 상황에서 변경
    - 중지/재시작, DB 인스턴스 교체, AWS 점검, OS 패치 등등
    - 따라서 가능하면 DNS로 접근하는 것을 권장
- 일반적으로 Production DB는 프라이빗 서브넷에 두는 경우가 많음
    - 보안적으로 매우 뛰어나니까
- 그럼 접속은 어떻게 해?
    - 배스쳔 호스트(가 일반적)
    - Instance Connect Endpoint가 있을 경우 활용 가능(무료)
        - 3389 포트만 활용 가능, 즉 RDS를 3389로만 사용해야 함
    - VPN / Direct Connect 등으로 VPC와 연결

## Amazon RDS Authentication
- 일반적인 Username/Password
    - AWS Secrets Manager로 관리 가능
- IAM 인증
    - IAM을 활용해 임시 토큰을 생성하여(15분) RDS에 접속하는 방법
        - 이 때, IAM 인증을 허용해줄 유저를 DB에 넣어줘야 함
    - 토큰 생성을 위해서는 rds-db:connect 권한 필요
    - IAM 컨디션 활용 가능
        - 예 : "인턴은 type:devonly 태그가 붙은 RDS 인스턴스에 회사 아이피로 1월 1일부터 1월 16일까지 아침 8시부터 저녁 6시까지만 접속 가능"
- Kerberos

## 데모 - RDS 접속과 인증
- 목표
    1. 프라이빗 서브넷에 생성된 RDS에 베스쳔 호스트로 접속
    2. RDS에 IAM DB 인증을 활용하여 접속
- 순서
    1. 기본 VPC에 두 개의 프라이빗 서브넷 생성 (+라우팅 테이블)
    2. 프라이빗 서브넷에 RDS를 프로비전
    3. 베스쳔 호스트를 통해 접속
        - ID/Password
        - IAM DB 인증

### 실습 순서
1. VPC 생성
    - 프라이빗 서브넷 0개
    - VPC 엔드포인트 없음
    - 디폴트 보안그룹 수정
        - 인바운드 규칙 모든 트래픽, 모든 소스로 수정
2. 서브넷 생성
    - 가용영역 a
    - CIDR 10.0.32.0/20
    - 두번째 가용영역 d
    - CIDR 10.0.48.0/20
3. RDS 프로비전
    - 서브넷 그룹 생성
        - Private Subnet 두개 생성
    - 데이터베이스 생성
        - MySQL
        - 프리티어
        - 암호 설정
        - 퍼블릭엑세스 아니오
        - DB 서브넷 그룹 교체
        - 데이터베이스 인증 옵션 : 암호 및 IAM 데이터베이스 인증
4. 베스쳔 호스트 생성
    - 인스턴스 시작
        - 키페어 사용
        - 네트워크 설정
            - VPC 선택
            - 퍼블릭 IP 할당
            - 퍼블릭 서브넷
5. DB 엔드포인트 연결 시도 (WorkBench) : 불가능.
6. WorkBench에서 베스쳔 호스트로 연결해보자
    - Standard TCP/IP over SSH
    - SSH Hostname : Bastion Host DNS
    - username : ec2-user
    - keyfile 사용
    - mysql 호스트 네임 : 데이터베이스 엔드포인트
    - 비밀번호 입력
7. IAM DB 인증을 해보자
    - 쿼리에 입력
        - CREATE USER 'testuser' IDENTIFIED WITH AWSAuthenticationPlugin AS 'RDS'; 
    - 새 DB 생성 (new schema)
        - GRANT ALL PRIVILEGES ON db_my_test.* TO 'testuser'@'%';
    - DB 접속 토큰 생성
        - 클라우드 쉘 열기
        - aws rds generate-db-auth-token --hostname {rds_dns}  --port 3389 --region ap-northeast-2 --username testuser
    - 새로 연결
        - Username : testuser
        - Password : 방금 생성된 스트링
        - Advanced : Enable clear~
8. 리소스 정리 : DB 삭제, 베스쳔호스트 삭제, VPC 삭제